In [7]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import (r2_score, auc, roc_auc_score, balanced_accuracy_score, 
                             average_precision_score, confusion_matrix, roc_curve)
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
exec(open('./iraps_classifier.py').read())

## drug TL-1-85

In [22]:
rna = pd.read_csv('./Galaxy216-[Depmap_drug_rna-seq_ready__TL-1-85.csv].tabular', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP4-671G15.2 (ENSG00000273483),OR6R2P (ENSG00000273484),RP11-225H22.7 (ENSG00000273485),RP11-731C17.2 (ENSG00000273486),RP4-621B10.8 (ENSG00000273487),RP11-114I8.4 (ENSG00000273488),RP11-180C16.1 (ENSG00000273489),AP000230.1 (ENSG00000273492),RP11-80H18.4 (ENSG00000273493),TL-1-85
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.910733,0.0,0.443607,1.555816,0.250962,2.432959,3.174726,0.014355,0.000000,0.923845
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.356144,0.0,0.000000,0.748461,0.070389,1.570463,3.065228,0.111031,0.000000,0.919654
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.150560,0.0,0.565597,0.432959,0.000000,0.815575,1.454176,0.014355,0.000000,0.909441
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.000000,0.0,0.000000,0.176323,0.097611,0.475085,2.049631,0.084064,0.000000,0.956789
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.124328,0.0,0.263034,0.584963,0.000000,1.570463,1.970854,0.150560,0.000000,0.989145
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.124328,0.0,0.000000,0.344828,0.028569,0.411426,1.244887,0.000000,0.000000,0.931208
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.000000,0.0,0.214125,0.963474,0.000000,1.636915,1.978196,0.084064,1.495695,0.594526
7,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.124328,0.0,0.356144,0.389567,0.084064,1.400538,2.179511,0.201634,0.000000,0.867585
8,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.250962,0.0,0.176323,0.963474,0.000000,1.516015,3.451541,0.111031,0.000000,0.868506
9,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.275007,0.0,0.176323,0.695994,0.000000,0.799087,1.655352,0.000000,0.000000,0.958399


In [23]:
X, y = rna.iloc[:, :-1].values, rna.iloc[:, -1].values
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [24]:
#IRAPS
cv = OrderedKFold(10)
results = {}
test_indices = {}
fold=0
auc_scores = []
ap_scores = []
iraps_core = IRAPSCore(n_iter=100, positive_thres=-1, verbose=0, random_state=None)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7, fc_thres=0.1, discretize=-1)
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(iraps)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict(X_test)
    results[fold] = y_p

    y_t = y_test < estimator.discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    data1 = go.Scatter(
            x = y_test,
            y = 1- y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0, 1],
            mode = "lines",
            name = "Perfect",
    )
    
    data3 = go.Scatter(
            x = [estimator.discretize_value, estimator.discretize_value],
            y = [0, 2],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='COR to minority'
        ),
        title="Cross val predict"

    )
    fig = go.Figure(data=[data1, data2, data3], layout=layout)
    iplot(fig)
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))

AUC: 0.864286
AP: 0.466239


AUC: 0.985714
AP: 0.876190


AUC: 0.803571
AP: 0.262528


AUC: 0.929825
AP: 0.705556


AUC: 0.964912
AP: 0.542857


...

AUC: 0.934211
AP: 0.571429


AUC: 0.899123
AP: 0.384774


AUC: 0.978070
AP: 0.812500


AUC: 0.942982
AP: 0.583333


AUC Mean: 0.922375
AP Mean: 0.582470


In [25]:
# BinarizeTargetClassifier
cv = OrderedKFold(10)
results = {}
fold=0
kbest = SelectKBest(score_func=f_classif, k=2000)
classifier = XGBClassifier(random_state=0)
pip = Pipeline([('prep_1', kbest), ('estimator', classifier)])
estimator0 = BinarizeTargetClassifier(pip, z_score=-1)

auc_scores = []
ap_scores = []
b_accuracy_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(estimator0)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict_proba(X_test)
    results[fold] = y_p

    y_p = y_p[:, 1]
    y_t = y_test < estimator.discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    b_accuracy = balanced_accuracy_score(y_t, y_p>0.5)
    print("Balanced Accuracy: %f" %b_accuracy)
    b_accuracy_scores.append(b_accuracy)
    
    data1 = go.Scatter(
            x = y_test,
            y = y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0.5, 0.5],
            mode = "lines",
            name = "class threshold",
    )
    
    data3 = go.Scatter(
            x = [estimator.discretize_value, estimator.discretize_value],
            y = [0, 1],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='predic_proba'
        ),
        title="True vs. predict_proba"

    )
    fig = go.Figure(data=[data1, data2, data3], layout=layout)
    iplot(fig)
    
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))
print("b_accuracy Mean: %f" %(sum(b_accuracy_scores)/float(len(b_accuracy_scores))))

AUC: 0.867857
AP: 0.420893
Balanced Accuracy: 0.591071


AUC: 0.942857
AP: 0.698889
Balanced Accuracy: 0.591071


AUC: 0.825000
AP: 0.392089
Balanced Accuracy: 0.573214


AUC: 0.969298
AP: 0.840909
Balanced Accuracy: 0.750000


AUC: 0.942982
AP: 0.601326
Balanced Accuracy: 0.616228


...

AUC: 0.894737
AP: 0.567288
Balanced Accuracy: 0.741228


AUC: 0.855263
AP: 0.319762
Balanced Accuracy: 0.598684


AUC: 0.973684
AP: 0.622024
Balanced Accuracy: 0.491228


AUC: 0.956140
AP: 0.634921
Balanced Accuracy: 0.732456


AUC Mean: 0.915326
AP Mean: 0.563977
b_accuracy Mean: 0.618518


In [26]:
# BinarizeTargetRegressor
cv = OrderedKFold(10)
results = {}
fold=0
kbest = SelectKBest(score_func=f_classif, k=2000)
classifier = XGBRegressor(random_state=0)
pip = Pipeline([('prep_1', kbest), ('estimator', classifier)])
#estimator0 = BinarizeTargetClassifier(pip, z_score=-1)

auc_scores = []
ap_scores = []
r2_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(pip)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict(X_test)
    results[fold] = y_p
    
    r2 = r2_score(y_test, y_p)
    print("r2: %f" %r2)
    r2_scores.append(r2)

    y_p = 1 - y_p
    discretize_value = y_train.mean() - y_train.std()
    y_t = y_test < discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    data1 = go.Scatter(
            x = y_test,
            y = y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0.5, 0.5],
            mode = "lines",
            name = "class threshold",
    )
    
    data3 = go.Scatter(
            x = [discretize_value, discretize_value],
            y = [0, 1],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='predic_proba'
        ),
        title="True vs. predict_proba"

    )
    fig = go.Figure(data=[data1, data3], layout=layout)
    iplot(fig)
    
print("r2 Mean: %f" %(sum(r2_scores)/float(len(r2_scores))))
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))

r2: 0.261717
AUC: 0.760714
AP: 0.571603


r2: 0.297434
AUC: 0.996429
AP: 0.966667


r2: 0.304194
AUC: 0.907143
AP: 0.433333


r2: 0.369990
AUC: 0.885965
AP: 0.685714


r2: 0.525559
AUC: 0.991228
AP: 0.916667


...

r2: 0.120246
AUC: 0.798246
AP: 0.408672


r2: 0.466678
AUC: 0.837719
AP: 0.444444


r2: 0.209182
AUC: 0.868421
AP: 0.717803


r2: 0.106394
AUC: 0.859649
AP: 0.408333


r2 Mean: 0.291961
AUC Mean: 0.883972
AP Mean: 0.602467


## drug Trametinib

In [14]:
rna = pd.read_csv('./Galaxy190-[Trametinib.csv].tabular', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP4-671G15.2 (ENSG00000273483),OR6R2P (ENSG00000273484),RP11-225H22.7 (ENSG00000273485),RP11-731C17.2 (ENSG00000273486),RP4-621B10.8 (ENSG00000273487),RP11-114I8.4 (ENSG00000273488),RP11-180C16.1 (ENSG00000273489),AP000230.1 (ENSG00000273492),RP11-80H18.4 (ENSG00000273493),Trametinib
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.910733,0.0,0.443607,1.555816,0.250962,2.432959,3.174726,0.014355,0.000000,0.926079
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.356144,0.0,0.000000,0.748461,0.070389,1.570463,3.065228,0.111031,0.000000,0.774235
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.150560,0.0,0.565597,0.432959,0.000000,0.815575,1.454176,0.014355,0.000000,0.863258
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.000000,0.0,0.000000,0.176323,0.097611,0.475085,2.049631,0.084064,0.000000,0.916183
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.124328,0.0,0.263034,0.584963,0.000000,1.570463,1.970854,0.150560,0.000000,0.727632
5,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.000000,0.0,0.214125,0.963474,0.000000,1.636915,1.978196,0.084064,1.495695,0.958660
6,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.124328,0.0,0.356144,0.389567,0.084064,1.400538,2.179511,0.201634,0.000000,0.494596
7,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.250962,0.0,0.176323,0.963474,0.000000,1.516015,3.451541,0.111031,0.000000,0.734423
8,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.275007,0.0,0.176323,0.695994,0.000000,0.799087,1.655352,0.000000,0.000000,0.411010
9,4.724650,0.000000,6.957450,1.835924,3.682573,0.000000,0.454176,5.825277,3.918386,4.197708,...,0.238787,0.0,0.879706,0.887525,0.000000,1.070389,1.495695,0.000000,0.000000,0.403970


In [15]:
X, y = rna.iloc[:, :-1].values, rna.iloc[:, -1].values
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [17]:
#IRAPS
cv = OrderedKFold(10)
results = {}
test_indices = {}
fold=0
auc_scores = []
ap_scores = []
b_accuracy_scores = []
iraps_core = IRAPSCore(n_iter=100, positive_thres=-1, verbose=0, random_state=None)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7, fc_thres=0.1, discretize=-1)
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(iraps)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict(X_test)
    results[fold] = y_p

    y_t = y_test < estimator.discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    data1 = go.Scatter(
            x = y_test,
            y = 1- y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0, 1],
            mode = "lines",
            name = "Perfect",
    )
    
    data3 = go.Scatter(
            x = [estimator.discretize_value, estimator.discretize_value],
            y = [0, 2],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='COR to minority'
        ),
        title="Cross val predict"

    )
    fig = go.Figure(data=[data1, data2, data3], layout=layout)
    iplot(fig)
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))

AUC: 0.872910
AP: 0.656127


AUC: 0.765957
AP: 0.510668


AUC: 0.751773
AP: 0.522270


AUC: 0.854610
AP: 0.561737


AUC: 0.744565
AP: 0.456145


...

AUC: 0.824275
AP: 0.605111


AUC: 0.764493
AP: 0.557942


AUC: 0.690217
AP: 0.341482


AUC: 0.775362
AP: 0.596447


AUC Mean: 0.764924
AP Mean: 0.510040


In [18]:
# BinarizeTargetClassifier
cv = OrderedKFold(10)
results = {}
fold=0
kbest = SelectKBest(score_func=f_classif, k=2000)
classifier = XGBClassifier(random_state=0)
pip = Pipeline([('prep_1', kbest), ('estimator', classifier)])
estimator0 = BinarizeTargetClassifier(pip, z_score=-1)

auc_scores = []
ap_scores = []
b_accuracy_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(estimator0)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict_proba(X_test)
    results[fold] = y_p

    y_p = y_p[:, 1]
    y_t = y_test < estimator.discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    b_accuracy = balanced_accuracy_score(y_t, y_p>0.5)
    print("Balanced Accuracy: %f" %b_accuracy)
    b_accuracy_scores.append(b_accuracy)
    
    data1 = go.Scatter(
            x = y_test,
            y = y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0.5, 0.5],
            mode = "lines",
            name = "class threshold",
    )
    
    data3 = go.Scatter(
            x = [estimator.discretize_value, estimator.discretize_value],
            y = [0, 1],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='predic_proba'
        ),
        title="True vs. predict_proba"

    )
    fig = go.Figure(data=[data1, data2, data3], layout=layout)
    iplot(fig)
    
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))
print("b_accuracy Mean: %f" %(sum(b_accuracy_scores)/float(len(b_accuracy_scores))))


AUC: 0.953177
AP: 0.845579
Balanced Accuracy: 0.709030


AUC: 0.861702
AP: 0.675851
Balanced Accuracy: 0.614362


AUC: 0.833333
AP: 0.611138
Balanced Accuracy: 0.801418


AUC: 0.870567
AP: 0.595339
Balanced Accuracy: 0.562057


AUC: 0.860507
AP: 0.733424
Balanced Accuracy: 0.708333


...

AUC: 0.835145
AP: 0.599216
Balanced Accuracy: 0.614130


AUC: 0.824275
AP: 0.635965
Balanced Accuracy: 0.666667


AUC: 0.724638
AP: 0.377219
Balanced Accuracy: 0.634058


AUC: 0.873188
AP: 0.735686
Balanced Accuracy: 0.717391


AUC Mean: 0.842820
AP Mean: 0.628574
b_accuracy Mean: 0.659810


In [21]:
# BinarizeTargetRegressor
cv = OrderedKFold(10)
results = {}
fold=0
kbest = SelectKBest(score_func=f_classif, k=2000)
classifier = XGBRegressor(random_state=0)
pip = Pipeline([('prep_1', kbest), ('estimator', classifier)])
#estimator0 = BinarizeTargetClassifier(pip, z_score=-1)

auc_scores = []
ap_scores = []
r2_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    test_indices[fold] = test_index
    
    estimator = clone(pip)
    estimator.fit(X_train, y_train)
    y_p = estimator.predict(X_test)
    results[fold] = y_p
    
    r2 = r2_score(y_test, y_p)
    print("r2: %f" %r2)
    r2_scores.append(r2)

    y_p = 1 - y_p
    discretize_value = y_train.mean() - y_train.std()
    y_t = y_test < discretize_value

    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    auc_scores.append(auc)
    
    ap = average_precision_score(y_t, y_p)
    print("AP: %f" %ap)
    ap_scores.append(ap)
    
    data1 = go.Scatter(
            x = y_test,
            y = y_p,
            mode = "markers",
            name = "True vs. Predicted",
    )
    fold += 1

    data2 = go.Scatter(
            x = [0, 1],
            y = [0.5, 0.5],
            mode = "lines",
            name = "class threshold",
    )
    
    data3 = go.Scatter(
            x = [discretize_value, discretize_value],
            y = [0, 1],
            mode = "lines",
            name = "Discretize",
    )

    layout = dict(
        xaxis=dict(
            title='Target'
        ),
        yaxis=dict(
            title='predic_proba'
        ),
        title="True vs. predict_proba"

    )
    fig = go.Figure(data=[data1, data3], layout=layout)
    iplot(fig)
    
print("r2 Mean: %f" %(sum(r2_scores)/float(len(r2_scores))))
print("AUC Mean: %f" %(sum(auc_scores)/float(len(auc_scores))))
print("AP Mean: %f" %(sum(ap_scores)/float(len(ap_scores))))

r2: 0.380357
AUC: 0.892977
AP: 0.714586


r2: 0.286745
AUC: 0.776596
AP: 0.454616


r2: 0.371800
AUC: 0.785461
AP: 0.436096


r2: 0.387635
AUC: 0.872340
AP: 0.618767


r2: 0.210861
AUC: 0.699275
AP: 0.512320


...

r2: 0.348068
AUC: 0.829710
AP: 0.522992


r2: 0.232844
AUC: 0.820652
AP: 0.577260


r2: 0.242839
AUC: 0.721014
AP: 0.505175


r2: 0.299577
AUC: 0.775362
AP: 0.506612


r2 Mean: 0.291049
AUC Mean: 0.784549
AP Mean: 0.517550
